# Create one big table with for each observation the summary of the status of the candidates

_Written: December 6, 2022_

In [24]:
from import_packages_generic import *
from import_functions_generic import *

In [25]:
L = time.localtime()
date = "{}-{}-{}".format(L[0],L[1],L[2],L[3],L[4],L[5])

saving_dir = 'tables/tables_v{}/'.format(date)
os.makedirs(saving_dir,exist_ok=True)

In [26]:
infolder = '/Users/desgranc/Documents/work/projects/HD120326/data/'

## Open the table listing the observations

In [68]:
path_csv_tables = infolder 

In [69]:
fn = 'obs_process1.csv'
dF_obs = pd.read_csv(path_csv_tables+fn, sep=';', comment='#')

dF_obs

,epoch (beginning night),mode,DIT (s),NDIT,nb cycles or NEXP,total exp time (min),nb frames,observing mode,PI,ESO_program,...,strehl (RTC),"seeing ("")",tau0 (ms),delta pa (deg),filter,polar,quality flag (1=good),quality reason,published,reduction status
0,2015-04-09,IRDIFS H23,16,13,1,3.5,13.0,pupil,Mickael & Francois,095.C-0607(A),...,/,1.10,3.3,NaN,H23,no,0,corono problem: il y avait Lyot a priori mais ...,0,1
1,2015-08-01,Zimpol I_PRIM fast pol,5,28,6,56.0,NaN,field,Booth & Julien,095.C-0487(A),...,NaN,NaN,NaN,NaN,NaN,yes,0,"fast polar, grade B redefined C afterwards LWE",0,to be done again
2,2016-04-05,IRDIFS H23,32,8,16,68.0,128.0,pupil,Mickael & Francois,097.C-0060(A),...,0.66000000000000003,1.10,3.5,37.0,H23,no,1,/,1,on Julien's hardware and also on the DC
3,2016-06-03,IRDIFS H23,32,5,16,43.0,80.0,pupil,Hinkley & Mattews,097.C-0949(A),...,0.70999999999999996,0.71,3.1,22.0,H23,no,1,/,0,to check
4,2016-06-13,IRDIS J23,64,5,16,85.0,80.0,pupil,GTO,097.C-0865(F),...,NaN,1.21,2.2,44.7,J23,no,0,/,0,relancé aujourd'hui
5,2018-06-01,IRDIS Polarimetry BB_H,64,2,40,NaN,80.0,field,Boccaletti,0101.C-0128(D),...,0.53000000000000003,0.45,4.2,NaN,BBH,yes,1,/,0,done with IRDAP
6,2019-06-26,IRDIFS BB H,32,14,4,30.0,56.0,pupil,Mickael & Francois,0101.C-0016(A),...,0.71999999999999997,1.84,2.4,36.0,BBH,no,0,"disk undetected, few frames",0,done
7,2019-07-09,IRDIFS BB H,32,14,16,119.0,224.0,pupil,Mickael & Francois,0101.C-0016(A),...,0.80000000000000004,0.54,3.1,58.0,BBH,no,1,"disk detected in PCA, process 307711)",0,done (use convert process 307407)


In [70]:
dF_obs.columns

Index(['epoch (beginning night)', 'mode', 'DIT (s)', 'NDIT',
       'nb cycles or NEXP', 'total exp time (min)', 'nb frames',
       'observing mode', 'PI', 'ESO_program', 'Grade', 'strehl (RTC)',
       'seeing (")', 'tau0 (ms)', 'delta pa (deg)', 'filter', 'polar',
       'quality flag (1=good)', 'quality reason', 'published',
       'reduction status'],
      dtype='object')

### Table LaTeX all epochs with candidates

Columns to use:

- Target, date, instru, filter, detection, status**

In [71]:
COLUMNS_TO_USE = ['epoch (beginning night)', 'mode', 
                  'nb frames', 'DIT (s)', #'grade ',
                  'delta pa (deg)','seeing (")', 'tau0 (ms)', 
                  'strehl (RTC)',  #'airmass avg'#, 'Strehl (RTC)', 'Strehl (measured)'
                 ]

round_up_value = [0,0,0,0,
                  0,2,0,2,
                 ]

print(len(round_up_value), 'vs', len(COLUMNS_TO_USE))

8 vs 8


In [72]:
dF = dF_obs[COLUMNS_TO_USE].copy()
nli, nco = np.shape(dF)
hline = '\hline'
# columns
ch = ''
for idx, col in enumerate(COLUMNS_TO_USE):
    ch += col 
    if idx < nco -1: ch += ' & '
    else: ch += ' \\\\'

print('\hline\hline\\noalign{\smallskip}')
print(ch)
print('\hline\hline\\noalign{\smallskip}')

# ORDER
#idx_sort = np.argsort(np.array(dF['details']))
#dF = dF.iloc[idx_sort]

for li in range(nli):
    ch = ''
    for co in range(nco):
        doit=0
        col = COLUMNS_TO_USE[co]
        value = dF.iloc[li,co]
        
        if col[:6] == 'Strehl':
            value*= 100

        if round_up_value[co] != 0: 
            #print('     ', COLUMNS_TO_USE[co], round_up_value[co],  dF.iloc[li,co])
            try: 
                if np.isnan(value): 
                    if col in ['e_radius_lit']: ch+= ''
                    else:  ch+= '$--$'
                else: doit = 1
            except:
                doit = 1
            if doit:
                nfig = round_up_value[co] 
                if nfig== -1: nfig=0
                try: 
                    #print('BOUH',nfig,value,type(value))
                    ch += '%.{}f'.format(nfig) % float(value)
                    #print('BOUH','%.{}f'.format(nfig) % value)
                except: 
                    ch += str(value)

        else: ch += str(value)
        if co == nco-1: ch += ' \\\\'
        elif co < nco-1 and col not in ['mass', 'radius_lit']: ch += ' & '
        elif co < nco-1 and col in ['mass']: ch += '\pm'
        elif co < nco-1 and col in ['radius_lit']: 
            if np.isnan(value): pass  
            else: ch += '\pm'
    print(ch)
print('\\noalign{\smallskip}\hline\hline') 

\hline\hline\noalign{\smallskip}
epoch (beginning night) & mode & nb frames & DIT (s) & delta pa (deg) & seeing (") & tau0 (ms) & strehl (RTC) \\
\hline\hline\noalign{\smallskip}
2015-04-09 & IRDIFS H23 & 13.0 & 16 & nan & 1.10 & 3.3 & / \\
2015-08-01 & Zimpol I_PRIM fast pol & nan & 5 & nan & $--$ & nan & $--$ \\
2016-04-05  & IRDIFS H23 & 128.0 & 32 & 37.0 & 1.10 & 3.5 & 0.66 \\
2016-06-03  & IRDIFS H23 & 80.0 & 32 & 22.0 & 0.71 & 3.1 & 0.71 \\
2016-06-13  & IRDIS J23 & 80.0 & 64 & 44.7 & 1.21 & 2.2 & $--$ \\
2018-06-01  & IRDIS Polarimetry BB_H & 80.0 & 64 & nan & 0.45 & 4.2 & 0.53 \\
2019-06-26 & IRDIFS BB H & 56.0 & 32 & 36.0 & 1.84 & 2.4 & 0.72 \\
2019-07-09 & IRDIFS BB H & 224.0 & 32 & 58.0 & 0.54 & 3.1 & 0.80 \\
\noalign{\smallskip}\hline\hline


In [60]:
value = 0.66000000000000003

In [61]:
'%.{}f'.format(nfig) % value

'0.66'

In [70]:
10**(1/-2.5)

0.3981071705534972

### Table LaTeX candidates most interesting

Columns to use:

- Target, date, instru, filter, detection, status**

In [293]:
dF_candidates.head()

,details,target,epoch,filter,coord_x_pix,coord_y_pix,ra_mas,ra_err_mas,ra_err_fit_mas,dec_mas,...,mag_abs_err_channel2,color,color_err,mag_lim_channel1,mag_lim_channel2,color_uplim,robust_detection_channel2,status_CMD,status_PM,status_CMD_PM
0,BD061339 2018-11-24 H23,BD061339,2018-11-24,H23,968.8,752.7,-2998.8,3.0,1.2,351.8,...,0.066,-0.002,0.093,15.925,15.889,-2.010,1.0,background,unknown,background (CMD)
1,BD061339 2018-11-24 H23,BD061339,2018-11-24,H23,392.0,876.0,4067.1,4.4,3.0,1861.6,...,0.096,-0.038,0.139,15.928,15.951,-1.083,1.0,background,unknown,background (CMD)
2,GJ176 2018-10-08 H23,GJ176,2018-10-08,H23,409.4,572.0,3853.6,3.3,0.8,-1861.8,...,0.071,-0.032,0.100,16.166,16.176,-3.173,1.0,background,unknown,background (CMD)
3,GJ229 2018-11-27 H23,GJ229,2018-11-27,H23,444.4,712.3,3425.0,2.9,1.0,-143.9,...,0.061,0.018,0.086,17.693,17.642,-1.819,1.0,background,unknown,background (CMD)
4,GJ229 2018-11-27 H23,GJ229,2018-11-27,H23,705.4,299.2,228.1,12.1,10.9,-5204.1,...,0.056,-1.761,0.079,17.886,17.922,-8.658,1.0,promising,unknown,promising (CMD)


In [294]:
dF_candidates.columns

Index(['details', 'target', 'epoch', 'filter', 'coord_x_pix', 'coord_y_pix',
       'ra_mas', 'ra_err_mas', 'ra_err_fit_mas', 'dec_mas',
       ...
       'mag_abs_err_channel2', 'color', 'color_err', 'mag_lim_channel1',
       'mag_lim_channel2', 'color_uplim', 'robust_detection_channel2',
       'status_CMD', 'status_PM', 'status_CMD_PM'],
      dtype='object', length=151)

#### Set the condition to select interesting candidates

In [317]:
dF = dF_candidates.copy()
cond_promising = dF['status_CMD_PM']=='promising (CMD)'
cond_promising_but = dF['status_CMD_PM']=='promising (CMD) but unredetected (PM)'
cond_unknown = dF['status_CMD_PM']=='unredetected (PM) + unknown (CMD)'
cond_interesting = np.logical_or(cond_promising, np.logical_or(cond_promising_but, cond_unknown))

dF = dF[cond_interesting].copy()

In [318]:
#np.array(dF.columns)

In [321]:
COLUMNS_TO_USE = ['target', 'epoch', 'instrument', 'filter', 
                  'ra_mas', 'ra_err_mas', 'dec_mas', 'dec_err_mas',
                  'sep_mas', 'sep_err_mas', 'pa_deg', 'pa_err_deg',
                  'contrast_channel1', 'contrast_err_channel1',
                  'contrast_channelallcomb', 'contrast_err_channelallcomb',
                  #'mag_channel1', 'mag_err_channel1',
                  'snr_channel1',
                  'snr_channelallcomb',
                  'status_CMD_PM'
                 ]

round_up_value = [0,0,0,0,
                    1,1,1,1, 1,1,1,1,
                    0,0,0,0, 1,1, 
                    0]


In [324]:
dF = dF[COLUMNS_TO_USE].copy()
nli, nco = np.shape(dF)
hline = '\hline'
# columns
ch = ''
for idx, col in enumerate(COLUMNS_TO_USE):
    ch += col 
    if idx < nco -1: ch += ' & '
    else: ch += ' \\\\'

print('\hline\hline\\noalign{\smallskip}')
print(ch)
print('\hline\hline\\noalign{\smallskip}')

# ORDER
#idx_sort = np.argsort(np.array(dF['details']))
#dF = dF.iloc[idx_sort]


COLS_WITH_ERR = ['ra_mas', 'dec_mas', 'sep_mas', 'pa_deg',
                 'mag_channel1', 'contrast_channel1']
COLS_NANS_POSSIBLE =  ['contrast_channel1', 'contrast_err_channel1', 'mag_channel1', 'mag_err_channel1']

for li in range(nli):
    ch = ''
    for co in range(nco):
        doit=0
        col = COLUMNS_TO_USE[co]
        value = dF.iloc[li,co]
        
        if value == 'H23': value = 'H2'
        elif value == 'K12': value = 'K1'
        
        if 0: #col in COL_NANS_POSSIBLE:
            if np.isnan(value): pass
            else: ch+= value
                
        else:    
            if round_up_value[co] != 0: 
                #print('     ', COLUMNS_TO_USE[co], round_up_value[co],  dF.iloc[li,co])
                try: 
                    if np.isnan(value): 
                        if col in ['e_radius_lit']: ch+= ''
                        else:  ch+= '$--$'
                    else: doit = 1
                except:
                    doit = 1
                if doit:
                    try: 
                        ch += '%.0f' % value 
                        #'%.{}f'.format(round_up_value[co]) % value
                        #if col == 'sep_mas': print(ch, round_up_value[co])
                    except: 
                        ch += str(value)

            else: ch += str(value)
        if co == nco-1: ch += ' \\\\'
        elif co < nco-1 and col not in COLS_WITH_ERR: ch += ' & '
        else: ch += '\pm'
            
        #elif co < nco-1 and col in ['mag_channel1', 'contrast_channel1']: 
        #    if np.isnan(value): pass  
        #    else: ch += '\pm'
    print(ch)
print('\\noalign{\smallskip}\hline\hline')

\hline\hline\noalign{\smallskip}
target & epoch & instrument & filter & ra_mas & ra_err_mas & dec_mas & dec_err_mas & sep_mas & sep_err_mas & pa_deg & pa_err_deg & contrast_channel1 & contrast_err_channel1 & contrast_channelallcomb & contrast_err_channelallcomb & snr_channel1 & snr_channelallcomb & status_CMD_PM \\
\hline\hline\noalign{\smallskip}
GJ229 & 2018-11-27 & IRDIS & H2 & 228\pm12 & -5204\pm3 & 5209\pm12 & 177\pm1 & 0.000197018\pm1.015e-05 & nan & nan & 88 & $--$ & promising (CMD) \\
HD85512 & 2019-11-25 & IRDIS & K1 & 911\pm6 & 11\pm4 & 912\pm6 & 89\pm0 & 4.494e-06\pm2.1745e-06 & nan & nan & 6 & $--$ & unredetected (PM) + unknown (CMD) \\
GJ674 & 2021-07-04 & IFS & YJH & -320\pm3 & 198\pm3 & 377\pm3 & 302\pm0 & nan\pmnan & 1.281e-05 & 1.4568e-06 & $--$ & 11 & unredetected (PM) + unknown (CMD) \\
GJ832 & 2017-05-27 & IFS & YJ & 487\pm3 & 194\pm3 & 524\pm3 & 68\pm0 & nan\pmnan & 2.678e-06 & 8.848e-07 & $--$ & 4 & unredetected (PM) + unknown (CMD) \\
\noalign{\smallskip}\hline\h

In [323]:
np.sqrt(5204**2+228**2)

5208.992224989398

## Generate Table "Target properties summary"

In [356]:
dF = dF_target_props.copy()

In [357]:
dF.columns

Index(['target name', 'distance', 'e_distance', 'parallax', 'e_parallax',
       'mag star J', 'mag star H', 'mag star K', 'age_sergio', 'e_age_sergio',
       'age_lit', 'age_min_lit', 'age_max_lit', 'spectral type', 'debris_disk',
       'multiple', 'mass', 'e_mass', 'mass_lit', 'e_mass_lit', 'ref_mass',
       'spec_type', 'ref_spec_type', 'ref_age', 'radius_lit', 'e_radius_lit',
       'ref_radius', 't_eff', 'e_t_eff', 'log_g', 'e_log_g', 'Vt', 'e_Vt',
       'FeH', 'e_FeH', 'Spectrum', 'Spectrum_SNR', 'Super-Earth confirmed',
       'Super-Earth unconfirmed', 'Sub-Saturn confirmed',
       'Sub-Saturn unconfirmed'],
      dtype='object')

In [361]:
COL_NAMES = ['Super-Earth confirmed', 'Super-Earth unconfirmed', 'Sub-Saturn confirmed', 'Sub-Saturn unconfirmed']
display_level2 = 0
for col_name in COL_NAMES:
    print('\n\n== Consider the column: {}. =='.format(col_name))
    if display_level2: print('\nValues are:', np.array(dF[col_name]))
    cond = dF[col_name] > 0
    dF_cond = dF[cond]
    N_cond = len(dF_cond)
    N_tot = len(dF)
    print('\nNumber of systems with {}: {}/{}.'.format(col_name.replace('_','-'), N_cond, N_tot))
    print('\nSystems without a priori a {}:\n'.format(col_name.replace('_','-')), np.array(dF[col_target_name][~cond]) )



== Consider the column: Super-Earth confirmed. ==

Number of systems with Super-Earth confirmed: 20/27.

Systems without a priori a Super-Earth confirmed:
 ['BD061339' 'GJ229' 'GJ649' 'GJ832' 'HD102365' 'HD3651' 'HD99492']


== Consider the column: Super-Earth unconfirmed. ==

Number of systems with Super-Earth unconfirmed: 11/27.

Systems without a priori a Super-Earth unconfirmed:
 ['61Vir' 'GJ176' 'GJ229' 'GJ3998' 'GJ422' 'GJ433' 'GJ674' 'GJ682'
 'HD136352' 'HD154088' 'HD189567' 'HD3651' 'HD38858' 'HD69830' 'HD85512'
 'HD99492']


== Consider the column: Sub-Saturn confirmed. ==

Number of systems with Sub-Saturn confirmed: 24/27.

Systems without a priori a Sub-Saturn confirmed:
 ['GJ649' 'GJ832' 'HD102365']


== Consider the column: Sub-Saturn unconfirmed. ==

Number of systems with Sub-Saturn unconfirmed: 14/27.

Systems without a priori a Sub-Saturn unconfirmed:
 ['GJ176' 'GJ229' 'GJ3998' 'GJ422' 'GJ674' 'GJ682' 'HD136352' 'HD154088'
 'HD189567' 'HD3651' 'HD69830' 'HD85512' 'H

### Create the summary table

In [441]:
COL_NAMES = ['Super-Earth confirmed', 'Sub-Saturn confirmed', 'debris_disk', 'multiple']
COL_COND = ['all the systems', 'systems with confirmed sub-Saturn', 'systems with confirmed super-Earth']
display_level1, display_level2 = 1, 0

n_names, n_conditions = len(COL_NAMES), len(COL_COND)
M = np.zeros((n_conditions, 1+n_names))

dF = dF_target_props.copy()

for idx_cond, col_cond in enumerate(COL_COND):
    print('\n\n== Consider {} systems =='.format(col_cond))
    if col_cond == 'systems with confirmed sub-Saturn': col_cond = 'Sub-Saturn confirmed' 
    elif col_cond == 'systems with confirmed super-Earth': col_cond = 'Super-Earth confirmed'
        
    if col_cond != 'all the systems': 
        cond_main =  dF[col_cond] > 0
    else: 
        cond_main = [True]*len(dF)
    dF_cond_main = dF[cond_main]
    N_cond_main = len(dF_cond_main)
    M[idx_cond,0] = N_cond_main
        
    if display_level1: print('The systems are:', np.array(dF_cond_main[col_target_name]))
    
    for idx_name, col_name in enumerate(COL_NAMES):
        if display_level1: print('\n-- Consider the column: {}. --'.format(col_name))
        if display_level2: print('\nValues are:', np.array(dF_cond_main[col_name]))
        
        if col_name in ['Super-Earth confirmed', 'Sub-Saturn confirmed']:
            cond_subsample = dF_cond_main[col_name] > 0
        else:
            cond_subsample = dF_cond_main[col_name] == True
        
        cond = np.logical_and(cond_main, cond_subsample)
        dF_cond = dF_cond_main[cond]
        
        N_cond = len(dF_cond)
        N_tot = len(dF)
        if display_level1: 
            print('\nNumber of {} systems with {}: {}/{}.'.format(col_cond, col_name.replace('_','-'), N_cond, N_cond_main))
            print('\nSystems with a priori a {}:\n'.format(col_name.replace('_','-')), 
              np.array(dF_cond_main[col_target_name][cond_subsample]) )
        
        M[idx_cond,1+idx_name] = N_cond



== Consider all the systems systems ==
The systems are: ['61Vir' 'BD061339' 'GJ163' 'GJ176' 'GJ180' 'GJ229' 'GJ3998' 'GJ422'
 'GJ433' 'GJ581' 'GJ649' 'GJ667C' 'GJ674' 'GJ682' 'GJ832' 'HD102365'
 'HD136352' 'HD154088' 'HD189567' 'HD20794' 'HD3651' 'HD38858' 'HD40307'
 'HD69830' 'HD85512' 'HD99492' 'Wolf1061']

-- Consider the column: Super-Earth confirmed. --

Number of all the systems systems with Super-Earth confirmed: 20/27.

Systems with a priori a Super-Earth confirmed:
 ['61Vir' 'GJ163' 'GJ176' 'GJ180' 'GJ3998' 'GJ422' 'GJ433' 'GJ581' 'GJ667C'
 'GJ674' 'GJ682' 'HD136352' 'HD154088' 'HD189567' 'HD20794' 'HD38858'
 'HD40307' 'HD69830' 'HD85512' 'Wolf1061']

-- Consider the column: Sub-Saturn confirmed. --

Number of all the systems systems with Sub-Saturn confirmed: 24/27.

Systems with a priori a Sub-Saturn confirmed:
 ['61Vir' 'BD061339' 'GJ163' 'GJ176' 'GJ180' 'GJ229' 'GJ3998' 'GJ422'
 'GJ433' 'GJ581' 'GJ667C' 'GJ674' 'GJ682' 'HD136352' 'HD154088' 'HD189567'
 'HD20794' 'HD3651'

/var/folders/k2/sfv4r4z938l77n1t4ydth2v401phl2/T/ipykernel_15676/3040692636.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dF_cond = dF_cond_main[cond]


In [446]:
COL_NAMES

['Super-Earth confirmed', 'Sub-Saturn confirmed', 'debris_disk', 'multiple']

In [443]:
M

array([[27., 20., 24.,  7.,  5.],
       [24., 20., 24.,  6.,  4.],
       [20., 20., 20.,  6.,  1.]])

In [451]:
## Add the columns in the DataFrame summary_target_properties.csv
dF_summary = pd.DataFrame([])

dF_summary.insert(0,'systems considered', COL_COND, False)
dF_summary.insert(1, 'total', M[:,0].astype(int), False)
for count, col_name in enumerate(COL_NAMES):
    dF_summary.insert(2+count, col_name, M[:,1+count].astype(int), False)

dF_summary.head()

,systems considered,total,Super-Earth confirmed,Sub-Saturn confirmed,debris_disk,multiple
0,all the systems,27,20,24,7,5
1,systems with Sub-Saturn confirmed,24,20,24,6,4
2,systems with Super-Earth confirmed,20,20,20,6,1


In [439]:
dF_summary.to_csv(saving_dir+'target_properties_summary.csv')

#### LaTeX

In [452]:
dF = dF_summary.copy()
nli, nco = np.shape(dF)
hline = '\hline'
# columns
ch = ''
for idx, col in enumerate(dF.columns):
    ch += col 
    if idx < nco -1: ch += ' & '
    else: ch += ' \\'

print('\hline\hline\\noalign{\smallskip}')
print(ch)
print('\hline\hline\\noalign{\smallskip}')

for li in range(nli):
    ch = ''
    for co in range(nco):
        ch += str(dF.iloc[li,co])
        if co < nco -1: ch += ' & '
        else: ch += ' \\'
    print(ch)
print('\\noalign{\smallskip}\hline\hline')


\hline\hline\noalign{\smallskip}
systems considered & total & Super-Earth confirmed & Sub-Saturn confirmed & debris_disk & multiple \
\hline\hline\noalign{\smallskip}
all the systems & 27 & 20 & 24 & 7 & 5 \
systems with Sub-Saturn confirmed & 24 & 20 & 24 & 6 & 4 \
systems with Super-Earth confirmed & 20 & 20 & 20 & 6 & 1 \
\noalign{\smallskip}\hline\hline


In [399]:

dF_summary.to_csv(saving_dir+'target_properties_summary_latex.csv',sep='&')

## Count the number of planets

Add the columns 

# Bin 

In [ ]:
# Cross-match long way...
list_targets = np.array(dF_mass['target name'])
print('List of the targets:\n', list_targets)
# For each target, gather the information
dF_crossmatch = []
dF1, dF2 = dF_mass, dF_names
col_target_name = 'target name'
for target in list_targets:
    cond_dF1 = dF1[col_target_name] == target
    cond_dF2 = dF2[col_target_name] == target
    dF_crossmatch.append(pd.concat([dF1[cond_dF1], dF2[cond_dF2]],axis=1))